In [177]:
import pandas as pd
import numpy as np
import xarray as xr
from configparser import ConfigParser
import sys
import os
import netCDF4 as nc

sys.path.append('../application')
from BMNP import BMNP

In [178]:
def ncExists(lat, lon):
    # Check to see if file exists at following location
    ncloc = './templates/mur_mmm.nc'
    
    # If it doesn't, we create it.
    if not BMNP.fileExists(ncloc):
        # Create an nc file with name mur_mmm
        newnc = nc.Dataset(ncloc, 'w', format='NETCDF4')
        
        # Create dimensions
        newnc.createDimension('time', None)
        newnc.createDimension('lat', len(lat))
        newnc.createDimension('lon', len(lon))
        
        # Create variables
        newnc.createVariable('time', 'f8', ('time',))
        newnc.createVariable('lat', 'f8', ('lat',))
        newnc.createVariable('lon', 'f8', ('lon',))
        newnc.createVariable('mmm_sst', 'f8', ('time', 'lat', 'lon'))
        
        # Add lat and lon data to variables
        newnc.variables['lat'][:] = lat
        newnc.variables['lon'][:] = lon
        
        # Add time data to variable
        newnc.variables['time'].units = 'Month and Year Value'
        
        # Create the file at location
        newnc.close()

In [179]:
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

def after2020data(location):
    # Read Data into dataframe, making top row the column names, index leftmost column
    df = pd.read_csv(location, header=0, index_col=0)
    year = location[-12:-8]
    
    # load the following nc file at this location:
    latlon_nc = nc.Dataset('../data/refined/MUR/2020/01/20200101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04_subsetted.nc4', 'r')
    
    # Get lat and lon data from nc file
    lat = latlon_nc.variables['lat'][:]
    lon = latlon_nc.variables['lon'][:]
    
    # Check to see if nc file exists.
    ncExists(lat, lon)
    
    # For loop cycling through months
    for month in months:
        monthyear = f'{year}{str(month)}'
        
        days = os.listdir(f'../data/refined/MUR/{year}/{month}')
        
        # Create empty numpy array named mmm
        mmm = []
        
        for day in days:
            file_loc = f'../data/refined/MUR/{year}/{month}/{day}'
            
            # Load in data from nc file
            ncfile = nc.Dataset(file_loc)
            
            # Extract "sst" and load this into a list.
            sst = ncfile.variables['sst'][:]
            
            # Make this a numpy array
            sst = np.array(sst)

            # If mmm np array is empty, make it equal to sst, otherwise add
            if len(mmm) == 0:
                mmm = sst
            else:
                mmm = mmm + sst
        
        # Divide mmm by the number of days in the month
        mmm = mmm / len(days)
        
        # Open nc file, the data mmm is meant to be in the shape of the lat and lon, we want time to be a single value, equal to the monthyear.
        ncfile = nc.Dataset('./templates/mur_mmm.nc', 'a')
        #ncfile.variables['mmm_sst'][:, :, :] = np.nan
        # Open nc file, the data mmm is meant to be in the shape of the lat and lon, we want time to be a single value, equal to the monthyear.
        ncfile = nc.Dataset('./templates/mur_mmm.nc', 'a')

        # Get the current number of time stamps
        num_time_stamps = len(ncfile.dimensions['time'])

        # Create a new time stamp for the current monthyear
        ncfile.variables['time'][num_time_stamps] = monthyear

        # Add the new array of mmm values to the new time stamp
        ncfile.variables['mmm_sst'][num_time_stamps, :, :] = mmm

        # Close the nc file
        ncfile.close()

In [180]:
# Check to see if a file templates/coordinates.csv exists
file = 'templates/'

if not BMNP.fileExists(file):
    BMNP.createFolders(file)

In [181]:
# Folder Locations for MUR Data years 2020-2022
murloc = '../data/csv/MUR/'

# Load in files in this location.
files = os.listdir(murloc)
delete = True
    
# If delete = True, delete the nc
if delete and BMNP.fileExists('./templates/mur_mmm.nc'):
    os.remove('./templates/mur_mmm.nc')
    
    
for i in range(len(files)):
    after2020data(f'{murloc}{files[i]}')
    
# We now open the nc file at the following location
ncfile = nc.Dataset('../data/refined/MUR_SST-Pre2020.nc')